# **Librerias y frameworks**

In [1]:
import requests
import pandas as pd
import numpy as np
import psycopg2
from dotenv import load_dotenv
import os
token = ''
load_dotenv("config.env")
token = os.getenv("TOKEN")

# **E- Extracción**

In [13]:
def dload_bmx_serie(serie,fechaInicio, fechaFin, token):
    '''Esta funcion devuelve un DataFrame extraido a través de la API del banco de México
     tomando como parametros el nombre de la serie y el rango de fechas. 
     El catalogo adjuntado previamente otorga los códigos alimentar la función'''
        
    url = 'https://www.banxico.org.mx/SieAPIRest/service/v1/series/'+serie+'/datos/'+fechaInicio+'/'+fechaFin
    print(url)
    headers = {'Bmx-Token':token} #Asi lo establece la web
    response = requests.get(url, headers=headers)
    status = response.status_code
    if status != 200: #código de estado http 200 significa OK.
        return print(f'Error en la consulta, codigo {status}')
    rawData = response.json() #aca es donde trae la info.
    data = rawData['bmx']['series'][0]['datos'] #con el método de corchetes se extraen los elementos que se necesitan de response.
    df = pd.DataFrame(data)
    df['dato'] = df['dato'].replace('N/E', np.nan).str.replace(',', '').astype(float)
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df.set_index('fecha', inplace=True)
    df.rename(columns={'dato': serie}, inplace=True)
    return df



### Descarga y creación de DF:

In [14]:
# Definir la lista de series a descargar
series = ['SF44042','SF44043','SF44044','SF43695','SF43702','SF43696']

df_final = pd.DataFrame()

# Parámetros comunes para todas las descargas
fechaInicio = ''
fechaFin = ''
token = token 

for serie in series:
    df = dload_bmx_serie(serie, fechaInicio, fechaFin, token)
    if df_final.empty:
        df_final = df
    else:
        df_final = pd.concat([df_final, df], axis=1)

print(df_final)

https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF44042/datos//
https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF44043/datos//
https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF44044/datos//
https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43695/datos//
https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43702/datos//
https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43696/datos//
            SF44042  SF44043  SF44044    SF43695    SF43702  SF43696
fecha                                                               
1989-01-01      NaN      NaN      NaN        NaN    14522.3      NaN
1989-01-02      NaN      NaN      NaN        NaN    14499.7      NaN
1989-01-03      NaN      NaN      NaN        NaN    14164.6      NaN
1989-01-04      NaN      NaN      NaN        NaN    13655.9      NaN
1989-01-05      NaN      NaN      NaN        NaN    13445.1      NaN
...             ...      ...      ...        ...        ...      ...
2023-07-16      

# **T- Transformación**

In [15]:
df_final.isna().sum()

SF44042    7678
SF44043    2922
SF44044    2922
SF43695    2556
SF43702       0
SF43696    2556
dtype: int64

In [16]:
df_final = df_final.fillna(0)

In [17]:
df_final.isna().sum()

SF44042    0
SF44043    0
SF44044    0
SF43695    0
SF43702    0
SF43696    0
dtype: int64

In [18]:
df_final.head()

,SF44042,SF44043,SF44044,SF43695,SF43702,SF43696
fecha,,,,,,
1989-01-01,0.0,0.0,0.0,0.0,14522.3,0.0
1989-01-02,0.0,0.0,0.0,0.0,14499.7,0.0
1989-01-03,0.0,0.0,0.0,0.0,14164.6,0.0
1989-01-04,0.0,0.0,0.0,0.0,13655.9,0.0
1989-01-05,0.0,0.0,0.0,0.0,13445.1,0.0


In [19]:
df_final.dtypes

SF44042    float64
SF44043    float64
SF44044    float64
SF43695    float64
SF43702    float64
SF43696    float64
dtype: object

In [20]:
df_final = df_final.reset_index()
df_final

,fecha,SF44042,SF44043,SF44044,SF43695,SF43702,SF43696
0,1989-01-01,0.0,0.0,0.0,0.0,14522.3,0.0
1,1989-01-02,0.0,0.0,0.0,0.0,14499.7,0.0
2,1989-01-03,0.0,0.0,0.0,0.0,14164.6,0.0
3,1989-01-04,0.0,0.0,0.0,0.0,13655.9,0.0
4,1989-01-05,0.0,0.0,0.0,0.0,13445.1,0.0
...,...,...,...,...,...,...,...
12614,2023-07-16,0.0,2715.3,2715.3,2715314.5,2714275.8,1038.7
12615,2023-07-17,0.0,2714.4,2714.4,2714398.1,2712107.7,2290.4
12616,2023-07-18,0.0,2712.3,2712.3,2712251.7,2711044.5,1207.2
12617,2023-07-19,0.0,2710.5,2710.5,2710473.4,2709888.7,584.7


In [ ]:
fechaInicio = input("Por favor, ingresa fecha de inicio de la carga en el formato AAAA-MM-DD: ")
fechaFin = input("Por favor, ingresa fecha de fin de la carga en el formato AAAA-MM-DD: ")

In [22]:
#El filtrado por fecha debe hacerse después.
df_final = df_final[(df_final['fecha'] >= fechaInicio) & (df_final['fecha'] <= fechaFin)]

In [23]:
df_final.head()

,fecha,SF44042,SF44043,SF44044,SF43695,SF43702,SF43696
12612,2023-07-14,0.0,2715.3,2715.3,2715314.5,2714275.8,1038.7
12613,2023-07-15,0.0,2715.3,2715.3,2715314.5,2714275.8,1038.7
12614,2023-07-16,0.0,2715.3,2715.3,2715314.5,2714275.8,1038.7
12615,2023-07-17,0.0,2714.4,2714.4,2714398.1,2712107.7,2290.4
12616,2023-07-18,0.0,2712.3,2712.3,2712251.7,2711044.5,1207.2


# **L- Carga**

### Conexión a Redshift:

In [31]:
host = os.getenv("HOST")
port = os.getenv("PORT")
user = os.getenv("USER")
password = os.getenv("PASSWORD")
database = os.getenv("DATABASE")

In [32]:
conn = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database,
)


### Creación de tabla vacia:

In [44]:
with conn.cursor() as cursor:
    create_table_query = '''
    CREATE TABLE jabrahanba_coderhouse.bancoMex (
        fecha DATE,
        SF44042 NUMERIC(16,2),
	    SF44043 NUMERIC(16,2),
	    SF44044 NUMERIC(16,2),
	    SF43695 NUMERIC(16,2),
	    SF43702 NUMERIC(16,2),
	    SF43696 NUMERIC(16,2)
    )
'''
    cursor.execute(create_table_query)
    conn.commit()


### Carga de tabla vacia:

In [47]:
filas_bancoMex = [tuple(df_final.iloc[i].values) for i in range (df_final.shape[0])]
fill_bancoMex = '''
INSERT INTO jabrahanba_coderhouse.bancoMex (fecha, SF44042, SF44043, SF44044, SF43695, SF43702, SF43696)
VALUES (%s, %s, %s, %s, %s, %s, %s)
'''
with conn.cursor() as cursor:
    cursor.executemany(fill_bancoMex, filas_bancoMex)
    conn.commit()